<a href="https://colab.research.google.com/github/hfarruda/deeplearningtutorial/blob/master/deepLearning_feedforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Feedforward networks

This example is part of the [*Deep Learning Tutorial*](https://github.com/hfarruda/deeplearningtutorial), authored by Henrique F. de Arruda, Alexandre Benatti, César Comin, and Luciano da Fontoura Costa.


## Binary Classification
This is the first example of deep learning implementation, in which we address binary classification of wine data. In this example, we consider one feedforward network with 5 hidden layers and with 30 neurons in each layer.  The provided networks were built only for a didactic purpose and are not appropriate for real applications.

First of all, we import the necessary libraries. Here we opt for using Keras (using TensorFlow backend).

In [0]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.vis_utils import plot_model
from keras.models import model_from_json
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

If you have a GPU, you can use the following code to allocate processing into it.  Otherwise, proceed to (*).

In [0]:
import tensorflow as tf 
from keras import backend as K

print(K.tensorflow_backend._get_available_gpus())

number_of_cpu_cores = 8
config = tf.ConfigProto(device_count = {'GPU': 1 , 'CPU': number_of_cpu_cores}) 
session = tf.Session(config=config) 
keras.backend.set_session(session)


Here, we use the Wine dataset. It is available at Sklearn library on [sklearn-datasets-wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html). For more information [wine-UCI](https://archive.ics.uci.edu/ml/datasets/Wine).
Because this dataset comprises three classes and here we exemplify a binary classification, we considered only the two first classes.



In [0]:
wine = load_wine()
data = wine['data']
target = wine['target']
target_names = wine['target_names'] 

hold = np.argwhere(target!=2).T[0]
data = data[hold]
target = target[hold]
target_names = target_names[0:1]

#Here, we divide our dataset into training and test sets.
test_size = 0.25 #fraction 
training_data,test_data,training_target,test_target = train_test_split(data, 
                                                    target, test_size=test_size)



In the following, we configure the neuronal network. It is not necessary to include bias because this parameter is set as true as default. 




In [0]:
#Set of parameters
input_dim = data.shape[1]
kernel_initializer = 'random_uniform'
bias_initializer='zeros'
activation_function_hidden = 'relu'
activation_function_output = 'sigmoid'
optimizer = 'adam'
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']
number_of_layers = 5
number_of_units_hidden = 30
number_of_units_output = 1
dropout_percentage = 0.25


#Creating model
ff_model = Sequential()
ff_model.add(Dense(units = number_of_units_hidden, 
                   activation = activation_function_hidden, 
                   kernel_initializer = kernel_initializer, 
                   input_dim = input_dim))

for i in range(number_of_layers-1):
  #Inserting a dense hidden layer
  ff_model.add(Dense(units = number_of_units_hidden, 
                     activation = activation_function_hidden, 
                     kernel_initializer = kernel_initializer, 
                     input_dim = number_of_units_hidden))
  #Inserting dropout
  ff_model.add(Dropout(dropout_percentage))

ff_model.add(Dense(units = number_of_units_output, 
                   activation = activation_function_output))
ff_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

W0830 18:53:06.774023 140522075584384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0830 18:53:06.824741 140522075584384 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0830 18:53:06.954720 140522075584384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0830 18:53:06.986447 140522075584384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_

In order to check the network topology, you can use the subsequent command.

In [0]:
ff_model.summary() 


Another option is to visualize the topology as a figure. 

In [0]:
#Saving the resultant figure as 'ff_model.png'.
plot_model(ff_model, to_file='ff_model.png', show_shapes=True, 
           show_layer_names=True)

The training step is executed as follows.

In [0]:
batch_size = 10
epochs = 200
ff_model.fit(training_data,training_target, batch_size = batch_size, 
             epochs = epochs)

In order to create an application, it is possible to save the network and the respective trained weights, as follows.



In [0]:
#Saving the network model
ff_model_json = ff_model.to_json()
with open('ff_model.json', 'w') as file:
    file.write(ff_model_json)

#Saving weights
ff_model.save_weights('ff_model.h5')

The following code can be employed to open a pre-trained model.

In [0]:
with open('ff_model.json', 'r') as file:
    ff_model_json = file.read()

ff_model = model_from_json(ff_model_json)
ff_model.load_weights('ff_model.h5')


There are different analysis that can account for the quality of the results. Here, to be didactic, we show only the measurement of accuracy. 

In [0]:
predictions = ff_model.predict(test_data)
#Because it is a binary classification, we consider the values higher than 0.5 
#as being part of class 1 otherwise 0.
predictions = (predictions > 0.5)
accuracy = accuracy_score(test_target, predictions)
print("Accuracy =", accuracy)

Accuracy = 0.9393939393939394


In this tutorial we only present a simple example of the neuronal network use. However, for a real application, it is necessary to consider a more elaborate analysis of the results, e.g., using a k-fold technique. 

##Multiclass Classification
In this example, we illustrate a multiclass classification through a wine dataset, in which there are three classes, which were defined according to their regions. We employed the same dataset presented above, but here we considered the three classes. To do so, we use the *softmax* activation function.

First of all, we import the necessary libraries. Here we opt for using Keras (using TensorFlow backend).

In [0]:
import numpy as np
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

If you have a GPU, you can use the following code to allocate processing into it.  Otherwise, proceed to (*).

In [0]:
import tensorflow as tf 
from keras import backend as K

print(K.tensorflow_backend._get_available_gpus())

number_of_cpu_cores = 8
config = tf.ConfigProto(device_count = {'GPU': 1 , 'CPU': number_of_cpu_cores}) 
session = tf.Session(config=config) 
keras.backend.set_session(session)

['/job:localhost/replica:0/task:0/device:GPU:0']


(*) In this examle the dataset used is Wine. It is available at Sklearn library on [sklearn-datasets-wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html). For more information [wine-UCI](https://archive.ics.uci.edu/ml/datasets/Wine).

These data show the results of a chemical analysis of wines grown in Italy, derived from three different cultivars in the same region, and can be loaded as follows.

In [0]:
wine = load_wine()
data = wine['data']
target = wine['target']
target_names = wine['target_names'] 

label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target)
target_one_hot_encoding = np_utils.to_categorical(target)

#Here, we divide our dataset into training and test sets.
test_size = 0.25 #fraction 
training_data,test_data,training_target,test_target = train_test_split(data, 
                                  target_one_hot_encoding, test_size=test_size)


In [0]:
np.sum(target==0), np.sum(target==1), np.sum(target==2)

(59, 71, 48)

In the following, we configure the neuronal network. It is not necessary to include bias because this parameter is set as true as default. 



In [0]:
#Set of parameters
input_dim = data.shape[1]
kernel_initializer = 'random_uniform'
bias_initializer='zeros'
activation_function_hidden = 'relu'
activation_function_output = 'softmax'
optimizer = 'adam'
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']
number_of_layers = 5
number_of_units_hidden = 30
number_of_units_output = len(set(target_names))
dropout_percentage = 0.25


#Creating model
ff_model = Sequential()
ff_model.add(Dense(units = number_of_units_hidden, 
                   activation = activation_function_hidden, 
                   kernel_initializer = kernel_initializer, 
                   input_dim = input_dim))

for i in range(number_of_layers-1):
  #Inserting a dense hidden layer
  ff_model.add(Dense(units = number_of_units_hidden, 
                     activation = activation_function_hidden, 
                     kernel_initializer = kernel_initializer, 
                     input_dim = number_of_units_hidden))
  #Inserting dropout
  ff_model.add(Dropout(dropout_percentage))

ff_model.add(Dense(units = number_of_units_output, 
                   activation = activation_function_output))
ff_model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
ff_model.summary() 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 30)                420       
_________________________________________________________________
dense_8 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout_6 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 30)                930       
_________________________________________________________________
dropout_7 (Dropout)          (None, 30)               

The training step is executed as follows.

In [0]:
batch_size = 10
epochs = 250
ff_model.fit(training_data,training_target, batch_size = batch_size,
             epochs = epochs)

Because there are three classes, we show the classification results through a confusion matrix.

In [0]:
predictions = ff_model.predict(test_data)

found_target = predictions.argmax(axis=1)
categorical_test_target = test_target.argmax(axis=1)

accuracy = accuracy_score(categorical_test_target, found_target)
print("Accuracy =", accuracy)

print("Confusion matrix:")
matrix = confusion_matrix(found_target,categorical_test_target)
print(matrix)